Lo primero es añadir al path la carpeta root del proyecto para poder acceder a los módulos y ejecutar aun así el notebook desde su carpeta

In [1]:
import sys
sys.path.insert(0, '..')

# Conexión y obtención general de reportes

In [2]:
from models import reports_collection as col
import controllers as c
from models import Report

In [3]:
reports = c.get_reports()
len(reports), reports[:5]

(40,
  <models.Report at 0x7f1a559f0070>])

In [4]:
masters = [r for r in reports if r.dupe_of is None]
len(masters), masters[:5]

(20,
  <models.Report at 0x7f1a559f00a0>])

In [5]:
duplicates = [r for r in reports if r.dupe_of is not None]
len(duplicates), duplicates[:5]

(20,
  <models.Report at 0x7f1a559f0100>])

# Generar pares maestro-maestro

In [6]:
from random import choice

master_pairs = []
while len(master_pairs) != 40: # TODO: Cambiar a 4000
    pair = (choice(masters), choice(masters))
    if pair not in master_pairs and pair[0] != pair[1]:
        master_pairs.append(pair)

In [7]:
from random import randint
i = randint(0, len(master_pairs)-1)
print(str(master_pairs[i][0]))
print(str(master_pairs[i][1]))
print(len(master_pairs))

Report 1431518 (dupe of None) - Image "copy to clipboard" produces the wrong colors, due to losing image's color profile
Report 1427224 (dupe of None) - Services.search.originalDefaultEngine is null on Linux Mint (breaking some non default engines since FF 57)
40


# Búsqueda de pares duplicado-maestro

In [8]:
fail = []
duplicate_pairs = []

for report in duplicates:
    try:
        master = c.get_report(report.dupe_of) # raises ReportNotFound
        duplicate_pairs.append((master, report))
    except Exception as e: # ReportNotFound
        fail.append(report)

In [9]:
print(f"[!] {len(fail)} reportes sin un maestro en BD ({len(fail) / len(duplicates) * 100:.2f}%)")
print(f"[+] {len(duplicate_pairs)} reportes con un maestro en BD ({len(duplicate_pairs) / len(duplicates) * 100:.2f}%)")
print(f"[+] {len(duplicate_pairs)+len(fail)}/{len(duplicates)} reportes duplicados analizados")

[!] 0 reportes sin un maestro en BD (0.00%)
[+] 20 reportes con un maestro en BD (100.00%)
[+] 20/20 reportes duplicados analizados


## Comprobar duplicados sin maestro aparente

Lo más probable es que sean un duplicado de un duplicado.

De esta forma, el dupe_of señala a otro reporte con un dupe_of que señala al verdadero maestro (puede tener más iteraciones)

In [10]:
from scraper.bugzilla_scraper import BugzillaBaseScraper

bugzilla = BugzillaBaseScraper()

scraped_masters = [] # (duplicado_sin_maestro_en_bd, maestro_scrapeado)
for report in fail:
    master = bugzilla.search_bug(report.dupe_of)
    scraped_masters.append((report, master))

In [11]:
new_pairs = []
for duplicate, master in scraped_masters:
    try:
        if master['dupe_of'] != None:
            master_db = c.get_report(master['dupe_of'])
            new_pairs.append((master_db, duplicate))
    except Exception as e: # ReportNotFound
        print(f"[!] El reporte maestro {master['id']} del duplicado {duplicate.report_id} no está en Base de datos y debería scrapearse")

len_new_pairs = str(len(new_pairs))
try:
    per_new_pairs = f"{len_new_pairs / len(scraped_masters) * 100:.2f}"
except:
    per_new_pairs = "0.00"
print(f"{len_new_pairs} reportes con un maestro en BD ({per_new_pairs}% de los reportes a analizar)")

0 reportes con un maestro en BD (0.00% de los reportes a analizar)


In [12]:
duplicate_pairs.extend(new_pairs)
print(f"Pares de maestro-duplicado: {len(duplicate_pairs)}")

Pares de maestro-duplicado: 20


# Resultados

In [13]:
print(f"Se tienen {len(masters)} reportes maestros")
print(f"Se tienen {len(duplicates)} reportes duplicados")
print(f"Total reportes en BD: {len(reports)}")
print()
print(f"Se obtuvieron {len(master_pairs)} pares maestro-maestro")
print(f"Se obtuvieron {len(duplicate_pairs)} pares maestro-duplicado")
print(f"Total pares: {len(master_pairs)+len(duplicate_pairs)}")

Se tienen 20 reportes maestros
Se tienen 20 reportes duplicados
Total reportes en BD: 40

Se obtuvieron 40 pares maestro-maestro
Se obtuvieron 20 pares maestro-duplicado
Total pares: 60


In [14]:
print(master_pairs[0][0])
print(master_pairs[0][1])

Report 1608461 (dupe of None) - Determine what kind of telemetry in-product Search Tips should collect
Report 1327551 (dupe of None) - Ctrl+Shift+T doesn't restore closed tab (100% reproducible)


In [15]:
print(duplicate_pairs[0][0])
print(duplicate_pairs[0][1])

Report 1598598 (dupe of None) - Intermittent mochitest <test-name> | application crashed [@ objc_retain + 0x1c]
Report 1607196 (dupe of 1598598) - Intermittent browser/components/customizableui/test/browser_976792_insertNodeInWindow.js (finished) | application terminated with exit code 2
